# Environment

#### Imports

In [1]:
import numpy as np
import pandas as pd
import cupy as cp
from tqdm.notebook import tqdm

In [4]:
!git clone https://github.com/sinc-lab/exp2GO.git

Cloning into 'exp2GO'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 107 (delta 25), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (107/107), 38.81 MiB | 17.61 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [5]:
path = 'exp2GO/data/'

# Data

In [6]:
species = 'ara'

#### Distance matrices

In [7]:
De = pd.read_csv(path + species + '_expression_dist_cosine.csv.zip', header=0, index_col=0) 

In [8]:
Dg = pd.read_csv(path + species + '_semantic_dist_BP_rel_min_exp.csv.zip', header=0, index_col=0)
Dg.index = Dg.index.str.upper()

In [9]:
# Filter non-annotated genes
Dg=Dg.dropna(axis=0, how='all')
Dg=Dg.dropna(axis=1, how='all') 

# Filter expression matrix according to GO matrix
De=De.filter(items=Dg.index, axis=0)
De=De.filter(items=Dg.index, axis=1) 

#### Annotations

In [10]:
ancestors = pd.read_csv(path + species + '_terms_gaf138_obo2016-06-01_BP_with_expr_EXP_anc.csv', header=None, index_col=0)
ancestors=ancestors.dropna(axis=0, how='all')
ancestors.fillna('',inplace=True)

labels = np.unique(ancestors.to_numpy().ravel())
#labels = np.trim_zeros(labels)
labels=labels[1:]

In [11]:
# change GO terms to numbers only in labs
labs=np.zeros_like(labels, dtype=np.uint64)
for i in range(len(labels)):
  labs[i]=int(labels[i][3:])
labels=labs

In [12]:
# change GO terms to numbers only in ancestors
anc = pd.DataFrame()
for gen in ancestors:
  labs=[]
  for i in range(len(ancestors[gen])):
    num=ancestors[gen][i][3:]
    if num != '':
      labs.append(int(num))
    else: 
      labs.append(0)
  anc[gen]=labs
anc.set_index(ancestors.index,inplace=True)
ancestors=anc

# NNMF d_GO reconstruction

#### Auxiliary functions

In [13]:
def send2back(d,idx):
  # sent to back these cols and rows in d
  # TO-DO warning for repeated genes (if len(idx)>len(unique(idx)))

  Nd = d.shape[0]
  Ns = idx.shape[0]

  idxinv = np.setdiff1d(np.arange(0,Nd),idx)
  idxinvidx = np.concatenate((idxinv, idx)) # new order
  
  dx = np.zeros(d.shape)

  dx[np.ix_(range(Nd-Ns),range(Nd-Ns))] = d[np.ix_(idxinv,idxinv)] # compacted left-up submatrix
  dx[np.ix_(range(Nd-Ns,Nd),range(Nd))] = d[np.ix_(idx,idxinvidx)] # last rows
  dx[np.ix_(range(Nd),range(Nd-Ns,Nd))] = d[np.ix_(idxinvidx,idx)] # last cols

  return dx

def zeroback(d, Ns):

  Nd = d.shape[0]
  d[np.ix_(range(Nd-Ns,Nd),range(Nd))] = 0;
  d[np.ix_(range(Nd),range(Nd-Ns,Nd))] = 0;

  return d

def return2future(db, idx):
  # restore last of db ows/cols to the idx positions
  # TO-DO warning for repeated genes (if len(idx)>len(unique(idx)))

  Nd = db.shape[0]
  Ns = idx.shape[0]

  idxinv = np.setdiff1d(np.arange(0,Nd),idx)
  idxinvidx = np.concatenate((idxinv, idx)) # new order
  
  dx = np.zeros(db.shape)

  dx[np.ix_(idxinv,idxinv)] = db[np.ix_(range(Nd-Ns),range(Nd-Ns))] # expand rows and cols
  dx[np.ix_(idx,idxinvidx)] = db[np.ix_(range(Nd-Ns,Nd),range(Nd))] # return last rows
  dx[np.ix_(idxinvidx,idx)] = db[np.ix_(range(Nd),range(Nd-Ns,Nd))] # return last cols

  return dx

#### NNMF CuPy version

In [16]:
def ssnmf_cp(De,Dg,Nc,lmbd,k,max_iter,tol=0.001):
  
  (m,n) = De.shape

  cp.random.seed(seed=333)
  A = cp.random.rand(m,k)
  S1 = cp.random.rand(k,n)
  S2 = cp.random.rand(k,n)

  p = m-Nc
  W = cp.zeros((m,n))
  W[cp.ix_(range(Nc),range(Nc))] = 1

  # learning the base
  e1ant = 1000
  e2ant = 1000
  cont = 0
  maxcont = 10

  l = 0
  cont = 0
  while (l<max_iter) and (cont<maxcont):
    A = A *( De @S1.T+  lmbd*(W *Dg)@S2.T)/((A@S1)@S1.T+  lmbd*(W *(A@S2))@S2.T+0.0000001)
    for u in range(k):
      A[:,u] = A[:,u]/cp.linalg.norm(A[:,u])
    S1 = S1 * ((A.T@De)/(A.T@(A@S1)+0.0000001))
    S2 = S2 * ((A.T@(W*Dg))/(A.T@(W*(A@S2))+0.0000001))
    err1 = cp.linalg.norm(De-A@S1,'fro')
    err2 = cp.linalg.norm(W*(Dg-A@S2),'fro')
    if (2-err1/e1ant-err2/e2ant)<tol:
      cont = cont+1
    else:
      cont = 0
    e1ant = err1
    e2ant = err2
    l = l+1
    #if l%100 == 0: print('l1: ',l)
    #if l==1: time0 = time.time()
    #if l==11: print('l1 10 time', time.time()-time0)

  miter = l        

  # full Dg estimation
  Dr1 = A@S2
  Dr2 = Dg.copy()
  m1 = cp.mean(Dg[0:Nc,0:Nc])
  m2 = cp.mean(Dr1[Nc:,:Nc])

  Dr2[cp.ix_(range(Nc,len(Dr2)),range(Nc))] = Dr1[cp.ix_(range(Nc,len(Dr1)),range(Nc))]
  Dr2 = ((1-W)*Dr2).T + Dr2

  # weigths redefinition
  W = cp.ones((m,n))
  W[cp.ix_(range(Nc,m),range(Nc,n))] = 0

  S2[cp.ix_(range(k),range(Nc,n))] = cp.ones((k,p)); #rand(k,p)
  eant = 1000
  cont = 0
  maxcont = 10
  tol = 0.00001

  l = 0
  cont = 0
  while (l<max_iter) and (cont<maxcont):
    S2 = S2*((A.T@(W*Dr2))/(A.T@(W*(A@S2))+0.0000001))
    err=cp.linalg.norm(W*(Dr2-A@S2),'fro')
    if abs(1-err/eant)<tol:
      cont = cont+1
    else:
      cont = 0
    eant=err
    l = l+1
    #if l%100 == 0: print('l2: ',l)

  aux = A@S2
  aux1 = aux[Nc:,Nc:]
  aux1 = (aux1.T+aux1)/2

  #Dr3 = Dr2
  Dr2[cp.ix_(range(Nc,len(Dr2)),range(Nc,len(Dr2)))] = aux1
  Dr2 = cp.minimum(Dr2,cp.max(Dg)*cp.ones(Dr2.shape))

  return Dr2

#### NNMF numpy version

In [15]:
def ssnmf(De,Dg,Nc,lmbd,k,max_iter):

  (m,n) = De.shape
  np.random.seed(seed=333)
  A = np.random.rand(m,k)
  S1 = np.random.rand(k,n)
  S2 = np.random.rand(k,n)

  p = m-Nc
  W = np.zeros((m,n))
  W[np.ix_(range(Nc),range(Nc))] = 1

  # learning the base
  e1ant = 1000
  e2ant = 1000
  cont = 0
  maxcont = 10
  tol = 0.001

  l = 0
  cont = 0
  while (l<max_iter) and (cont<maxcont):
    A = A *( De @S1.T+  lmbd*(W *Dg)@S2.T)/((A@S1)@S1.T+  lmbd*(W *(A@S2))@S2.T+0.0000001)
    for u in range(k):
      A[:,u] = A[:,u]/np.linalg.norm(A[:,u])
    S1 = S1 * ((A.T@De)/(A.T@(A@S1)+0.0000001))
    S2 = S2 * ((A.T@(W*Dg))/(A.T@(W*(A@S2))+0.0000001))
    err1 = np.linalg.norm(De-A@S1,'fro')
    err2 = np.linalg.norm(W*(Dg-A@S2),'fro')
    if (2-err1/e1ant-err2/e2ant)<tol:
      cont = cont+1
    else:
      cont = 0
    e1ant = err1
    e2ant = err2
    l = l+1
    if l%100 == 0: print('l1: ',l)
    if l==1: time0 = time.time()
    if l==11: print('l1 10 time', time.time()-time0)

  miter = l        
 
  # full Dg estimation
  Dr1 = A@S2
  Dr2 = Dg.copy()
  m1 = np.mean(Dg[0:Nc,0:Nc])
  m2 = np.mean(Dr1[Nc:,:Nc])

  Dr2[np.ix_(range(Nc,len(Dr2)),range(Nc))] = Dr1[np.ix_(range(Nc,len(Dr1)),range(Nc))]
  Dr2 = ((1-W)*Dr2).T + Dr2

  # weigths redefinition
  W = np.ones((m,n))
  W[np.ix_(range(Nc,m),range(Nc,n))] = 0

  S2[np.ix_(range(k),range(Nc,n))] = np.ones((k,p)); #rand(k,p)
  eant = 1000
  cont = 0
  maxcont = 10
  tol = 0.00001

  l = 0
  cont = 0
  while (l<max_iter) and (cont<maxcont):
    S2 = S2*((A.T@(W*Dr2))/(A.T@(W*(A@S2))+0.0000001))
    err=np.linalg.norm(W*(Dr2-A@S2),'fro')
    if abs(1-err/eant)<tol:
      cont = cont+1
    else:
      cont = 0
    eant=err
    l = l+1
    if l%100 == 0: print('l2: ',l)

  aux = A@S2
  aux1 = aux[Nc:,Nc:]
  aux1 = (aux1.T+aux1)/2

  #Dr3 = Dr2
  Dr2[np.ix_(range(Nc,len(Dr2)),range(Nc,len(Dr2)))] = aux1
  Dr2 = np.minimum(Dr2,np.max(Dg)*np.ones(Dr2.shape))

  return Dr2

#### NNMF-GO

In [17]:
def nnmfgo(d_expr, d_GO, idx_to_rec, lmbd, dim, max_iter,tol):

  # Format GO distance matrix (back and blank genes to annotate)
  dg = send2back(d_GO,idx_to_rec)
  dg = zeroback(dg,len(idx_to_rec))

  # Format expression distance matrix (back genes to annotate)
  de = send2back(d_expr,idx_to_rec)

  # NNMF
  num_genes_annotated = d_expr.shape[0]-len(idx_to_rec)

  # without cupy: 392 s for 10 iterations (aprox 10 h for 400 l1 + 500 l2)
  #rec = ssnmf(d_expr,d_GO,num_genes_annotated,lmbd,dim,max_iter)
  
  # with cupy: 11 s for 10 iterations (aprox 18 min for 400 l1 + 500 l2)
  d_GO_cp = cp.array(dg)  
  d_expr_cp = cp.array(de)
  rec_cp = ssnmf_cp(d_expr_cp,d_GO_cp,num_genes_annotated,lmbd,dim,max_iter,tol)
  rec = cp.asnumpy(rec_cp)
  #rec = cp.asnumpy(d_GO_cp)

  # Restore the original order in the distance matrix
  d_GO_rec = return2future(rec,idx_to_rec);

  return d_GO_rec

# Bayesian inference

In [19]:
def bayesprobs(d,idxB,labelsAwB,ancestors,pexp):
  # ancestors: GO terms for each known gene

  idxA = np.setdiff1d(np.arange(0,len(d)),idxB) # annotated genes
  
  weights = np.zeros((len(idxB),len(labelsAwB))) # % ~> likelihood p(gene|label)
  countAB = np.zeros((len(idxB),len(labelsAwB))) # % ~> prior p(label)

  for genB in range(len(idxB)):
    col = idxB[genB]
    dnorm_col = d[:,col] 
    dnorm_col = (dnorm_col-np.min(dnorm_col))/(np.max(dnorm_col)-np.min(dnorm_col)) 
    for genA in range(len(idxA)):
      labsA = ancestors.iloc[idxA[genA],:].dropna().to_numpy()
      labsA = np.setxor1d([0],labsA)
      row = idxA[genA]
      d_gen=d[row,col] 
      for t in range(len(labsA)):
        idxLab = np.where(labelsAwB == labsA[t])
        weights[genB,idxLab] = weights[genB,idxLab]+(2/(1+d_gen)-1)**pexp 
        countAB[genB,idxLab] = countAB[genB,idxLab]+1

  return (weights, countAB)

In [20]:
def bayesinf(genB,p,cumpcut,labelsAwB):

  prob = p[genB,:]
  prob = prob/np.sum(prob)
  ind = np.argsort(prob)
  indd = ind[::-1] # descendent sort
  cum = np.cumsum(prob[indd])
  pp = np.argmax(cum>cumpcut)
  inB = labelsAwB[indd[:pp+1]]
  
  return (inB, indd, cum)

In [21]:
# separated for otimization
def bayesinf_nocut(genB,p):

  prob = p[genB,:]
  prob = prob/np.sum(prob)
  ind = np.argsort(prob)
  indd = ind[::-1] # descendent sort
  cum = np.cumsum(prob[indd])
  
  return indd, cum

def bayesinf_justcut(cum, cumpcut, indd, labelsAwB):

  pp = np.argmax(cum>cumpcut)
  inB = labelsAwB[indd[:pp+1]]
  
  return inB

# Main LOO

#### Performance measures

In [22]:
def spf1(refP,predP,labels):
  predN = np.setdiff1d(labels,predP)
  refN = np.setdiff1d(labels,refP)
  TP = len(np.intersect1d(refP,predP))
  TN = len(np.intersect1d(refN,predN))
  FP = len(np.intersect1d(predP,refN))
  FN = len(np.intersect1d(predN,refP))
  sens = TP/(TP+FN)
  prec = TP/(TP+FP)
  if sens+prec>0: F1 = 2*sens*prec/(sens+prec)
  else: F1 = 0
  return (TP, TN, FP, FN, sens, prec, F1)


# find best F1
def F1max(indord,labels,refP,refN):
  maxF1 = 0
  maxcum = 0
  
  for i in range(len(indord)):
    predP = labels[indord[:i+1]]
    TP, TN, FP, FN, sens, prec, F1 = spf1(refP,predP,labels)

    if F1>maxF1:
      maxF1 = F1
      maxcum = cum[i]

  return (maxF1, maxcum)

### LOO

In [23]:
lmbd = 0.001    #@param {type:"slider", min:0, max:200000, step:0.05}
dim = 80        #@param {type:"slider", min:10, max:2000, step:10}
pexp = 6        #@param {type:"slider", min:1, max:15, step:1}
max_iter = 1000 #@param {type:"slider", min:1, max:5000, step:100}
tol = 0.001     #@param {type:"slider", min:0, max:1, step:0.001}

gamma = 1.0     #param {type:"slider", min:0, max:1, step:0.05}
cumpcut = 0.90  #param {type:"slider", min:0, max:1, step:0.05}

In [24]:
Ngenes = Dg.shape[0]
Nlabels = len(labels)

d_expr = De.to_numpy()
d_GO = Dg.to_numpy()

p = np.zeros((Ngenes,Nlabels))

for igenloo in tqdm(range(Ngenes)):
  idx_to_ann = np.array([igenloo])

  d_GO_rec = nnmfgo(d_expr, d_GO, idx_to_ann, lmbd, dim, max_iter, tol)
  #d_GO_rec = d_GO # <<< oracle
  
  (weights, countAB) = bayesprobs(d_GO_rec,idx_to_ann,labels,ancestors,pexp)

  p[igenloo,:] = weights*countAB

In [ ]:
indord = np.zeros((Ngenes, Nlabels), dtype=np.uint32)
cum =  np.zeros((Ngenes, Nlabels))
refP = np.zeros((Ngenes, len(ancestors.iloc[0,:].dropna().values)), dtype=np.uint32)

for igenloo in tqdm(range(Ngenes)):
  indord[igenloo,:], cum[igenloo,:] = bayesinf_nocut(igenloo, p)
  refP[igenloo,:] = ancestors.iloc[igenloo,:].dropna().values

F1th = np.zeros((Ngenes,))
ccut=0.80
for igenloo in range(Ngenes):
  labelsB_genB = bayesinf_justcut(cum[igenloo,:], ccut, indord[igenloo,:], labels)
  _, _, _, _, _, _, F1 = spf1(refP[igenloo,:],labelsB_genB,labels)
  F1th[igenloo] = F1

print(F1th)
print(np.mean(F1th))
print(np.std(F1th))